In [3]:
from compChem.Hartree_Fock import Molecule # package is available on github
from scipy.linalg import eigh, eig
import numpy as np
import psi4

In [4]:
numpy_memory = 4
psi4.set_memory(int(5e8))

500000000

In [38]:
class ConstrainedMolecule:
    def __init__(self, geom_file):
        """initiation method, will take in a geometry"""
        #this parameter contains all parameters and methods from the Molecule class
        self.id = Molecule(geom_file)
        self.id.setGuess()
        
    
    def setChargeDensity(self, new_matrix):
        """sets the pMatrix to a new value"""
        self.pMatrix = new_matrix
        
        
    def fock_Alternator(self, f_a, f_b):
        """
        imposes the constraint on two given fock matrices
        
        input:
        f_*: the fock matrices
        """
        #we need a closed shell type operator, it is already in the correct basis
        f_cs = (f_a + f_b)/2
        delta = (f_b - f_a)/2
        
        #amount of paired electrons if the number of alpha electrons is greater then or equal to the number of beta electrons
        cc = self.id.beta 
        
        #amount of unpaired electrons
        oo = self.id.alpha - self.id.beta
        
        dim = f_cs[:cc+oo, cc+oo:].shape
        #alter first blocks
        delta[:cc+oo, cc+oo:] = np.zeros(dim)
        delta[cc+oo:, :cc+oo] = np.zeros(dim).T
        
        
        return f_cs - delta, f_cs + delta
        
    
    def basischanger(self):
        """
        changes to NO basis, applies constraint, then changes back
        """
        # transform p to MO basis, where mo basis = the eigenfunctions of the f_a operator
        a = self.id.getDensityMatrix("alpha")
        b = self.id.getDensityMatrix("beta")
        f_a, f_b = self.id.displayFockMatrix("alpha"), self.id.displayFockMatrix("beta")
        p = (a+b)/2
        c = eigh(f_a, self.id.overlap)[1] # we only need the c matrix, not the eigenvalues themselves,
        
        # pay attention, c matrices are not unitary
        c_inv = np.linalg.inv(c) # we need the inverse for later
        p_trans = np.einsum("pq, qr, rs->ps", c.T, p, c, optimize=True)
        
        
        # transform the fock matrices to MO basis
        f_a_mo = np.einsum("pq, qr, rs->ps", c.T, f_a, c, optimize=True)
        f_b_mo = np.einsum("pq, qr, rs->ps", c.T, f_b, c, optimize=True)
        
        # transform the fock matrices to NO basis
        d = eigh(p_trans)[1]
        d = d[:, ::-1] #invert all collumns
        # d is unitary, so no worries here
        f_a_no = np.einsum("pq, qr, rs->ps", d.T, f_a_mo, d, optimize=True)
        f_b_no = np.einsum("pq, qr, rs->ps", d.T, f_b_mo, d, optimize=True)
        
        f_a_alt, f_b_alt = self.fock_Alternator(f_a_no, f_b_no)

        # transform back to AO basis
        f_a_mo = np.einsum("pq, qr, rs->ps",d, f_a_alt, d.T, optimize=True)
        f_a_ao = np.einsum("pq, qr, rs->ps",c_inv.T, f_a_mo, c_inv, optimize=True)
        f_b_mo = np.einsum("pq, qr, rs->ps",d, f_b_alt, d.T, optimize=True)
        f_b_ao = np.einsum("pq, qr, rs->ps",c_inv.T, f_b_mo, c_inv, optimize=True)
        
        return f_a_ao, f_b_ao
    
    
    def iteratinator(self):
        """adapted version of the original iterator function in the compChem.Hartree_Fock Molecule object"""
        # setting up entry parameters for the while loop
        E_new = 0  
        E_old = 0
        convergence = False

        # step 2: start iterating
        itercount = 0
        while not convergence and itercount < 5000:
            # calculating block: calculates energies
            E_new = self.id.getElectronicEnergy()
            E_total = self.id.getTotalEnergy()

            # generating block: generates new matrices UHF: account for alpha and beta, first we do the CUHF alteration
            f_a_n, f_b_n = self.basischanger()
            self.id.setGuess(f_a_n, "alpha")
            self.id.setGuess(f_b_n, "beta")

            # comparing block: will answer the "Are we there yet?" question
            if abs(E_old - E_new) < self.id.converge:
                convergence = True


            # maintenance block: keeps everything going
            print(f"iteration: {itercount}, E_tot: {E_total: .8f}, E_elek: {E_new: .8f}, deltaE: {E_new - E_old: .8f}")
            E_old = E_new

            itercount += 1
        
        return E_total

In [39]:
psi4.set_options({"BASIS": "cc-pvdz"})
h1 = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
h1.id.setConvergence(1e-12)
h1.iteratinator()

iteration: 0, E_tot: -1.33707541, E_elek: -2.92460704, deltaE: -2.92460704
iteration: 1, E_tot: -1.50331998, E_elek: -3.09085161, deltaE: -0.16624457
iteration: 2, E_tot: -1.50356122, E_elek: -3.09109285, deltaE: -0.00024124
iteration: 3, E_tot: -1.50337163, E_elek: -3.09090326, deltaE:  0.00018959
iteration: 4, E_tot: -1.50329114, E_elek: -3.09082277, deltaE:  0.00008048
iteration: 5, E_tot: -1.50325852, E_elek: -3.09079016, deltaE:  0.00003262
iteration: 6, E_tot: -1.50324454, E_elek: -3.09077617, deltaE:  0.00001398
iteration: 7, E_tot: -1.50323828, E_elek: -3.09076991, deltaE:  0.00000626
iteration: 8, E_tot: -1.50323540, E_elek: -3.09076703, deltaE:  0.00000288
iteration: 9, E_tot: -1.50323406, E_elek: -3.09076569, deltaE:  0.00000134
iteration: 10, E_tot: -1.50323343, E_elek: -3.09076507, deltaE:  0.00000063
iteration: 11, E_tot: -1.50323314, E_elek: -3.09076477, deltaE:  0.00000029
iteration: 12, E_tot: -1.50323300, E_elek: -3.09076464, deltaE:  0.00000014
iteration: 13, E_tot: 

-1.5032328821181995

In [11]:
psi4.energy('scf', molecule=h1.id.id)

-1.5050449496717984

In [ ]:
mol = psi4.geometry("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")
wfn = psi4.core.Wavefunction.build(mol, psi4.core.get_global_option('basis'))
e, wfn = psi4.energy('scf', molecule=mol, return_wfn=True)
psi4.core.CUHF.compute_energy(wfn)

In [27]:
f_a = h1.id.displayFockMatrix('alpha')
c = eigh(f_a, h1.id.overlap)[1]
a = h1.id.getDensityMatrix("alpha")
b = h1.id.getDensityMatrix("beta")
p = (a + b)/2
e = c.T.dot(p).dot(c)
f1, f = eigh(e)
f = f[:, ::-1]
f_trans = c.T.dot(f_a).dot(c)
f_no = f.T.dot(f_trans).dot(f)
q = f.T.dot(e).dot(f)
q[abs(q) < 1e-10] = 0
q

array([[8.46023993e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 3.79407217e-01, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.65155096e-03, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+0

This value does not seem correct (UHF gives about -1.506 H for the total energy). This might be the case because the Fock operator depends on the two electron integrals have not been transformed to the correct basis yet. Can this be the case?


In [13]:
psi4.set_options({"BASIS": "cc-pvdz"})
h_test = ConstrainedMolecule("""
H 0 0 0
H 0 0.86602540378 0.5
H 0 0 1
units angstrom""")

In [21]:
s = h_test.id.overlap
s_eigh = eigh(s)
s_diag = np.diag(s_eigh[0]**(-1/2))
x = s_eigh[1].dot(s_diag).dot(s_eigh[1].T)
x.T.dot(s).dot(x)

array([[ 1.00000000e+00, -9.35449634e-16, -5.06539255e-16,
         1.15513088e-17, -6.93889390e-18, -1.01307851e-15,
        -1.90819582e-16,  5.81999726e-16, -5.13331503e-18,
         3.74700271e-16,  3.46944695e-17, -1.06858966e-15,
         5.04804532e-16, -4.78284731e-18, -5.55111512e-17],
       [-9.04224612e-16,  1.00000000e+00, -4.82253126e-16,
        -1.40775426e-16, -6.65700134e-17,  1.34614542e-15,
         1.63497688e-16, -9.54748433e-16, -1.15689374e-18,
        -2.20309881e-16,  1.09287579e-16,  3.15719673e-15,
        -1.12236609e-15,  3.94550374e-17, -3.12250226e-16],
       [-5.44703171e-16, -4.86589935e-16,  1.00000000e+00,
         6.00991998e-17,  2.25514052e-16, -4.64905892e-16,
        -5.20417043e-16,  4.72712147e-16,  1.61936639e-17,
        -5.82867088e-16, -7.63278329e-17, -1.17961196e-16,
         6.01949046e-16,  4.57643834e-17,  8.04911693e-16],
       [ 1.15513088e-17, -1.40775426e-16,  6.00991998e-17,
         1.00000000e+00, -2.81355801e-15, -9.12145162

In [22]:
a = h_test.id.getDensityMatrix('alpha')
b = h_test.id.getDensityMatrix('beta')
x_i = np.linalg.inv(x)
x_i_t = np.linalg.inv(x.T)
a_trans = x_i.dot(a).dot(x_i_t)
b_trans = x_i.dot(b).dot(x_i_t)
p = (a_trans + b_trans)/2
c = eigh(p)
c

(array([-1.78194235e-17, -2.14727327e-18,  2.74455182e-34,  2.77327639e-34,
         2.01211142e-33,  7.38908619e-19,  2.41211690e-18,  8.05840392e-18,
         2.67748556e-17,  2.72177565e-17,  3.30018027e-17,  6.52023815e-17,
         4.44089210e-16,  5.00000000e-01,  1.00000000e+00]),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          6.16588592e-01,  6.31105094e-01, -4.70664284e-01],
        [-1.30640235e-01,  3.00844396e-01,  6.26751203e-16,
         -8.41143893e-16, -6.43008383e-16, -3.73948706e-02,
         -7.89667496e-02,  2.20038656e-01, -4.24872122e-01,
          2.57854156e-01, -3.49308280e-01, -4.09191964e-02,
         -5.36766110e-01,  3.19698737e-01, -2.74506401e-01],
        [-5.91552737e-02,  8.33329270e-02, -3.55718948e-16,
         -6.54985913e-17, -1.78970201e-16,  1.518

Up untill here everything should be correct (density matrices are idempotent, p matrix has correct eigenvalues). The area below is still under construction.

In [23]:
f_a = h_test.id.displayFockMatrix('alpha')
f_b = h_test.id.displayFockMatrix('beta')

In [24]:
f_a_trans = x.dot(f_a).dot(x.T)
f_a_no = c[1].T.dot(f_a_trans).dot(c[1])
f_b_trans = x.dot(f_b).dot(x.T)
f_b_no = c[1].T.dot(f_b_trans).dot(c[1])

NameError: name 'x' is not defined

This expression seems a bit too simple. Nevertheless, according to Tsuchimochi & Scuseria it should hold.

In [25]:
f_cs = (f_a_no + f_b_no)/2

In [26]:
a = h_test.id.getDensityMatrix("alpha")
b = h_test.id.getDensityMatrix("beta")
a - b

array([[ 5.97630536e-01,  4.08572328e-01,  5.95842407e-03,
        -1.83937606e-17, -8.13875443e-06, -2.98874857e-01,
        -2.04326902e-01, -5.15242734e-02, -7.21818209e-18,
         3.31883368e-02, -2.98755680e-01, -2.04245426e-01,
        -5.15319158e-02, -4.00369981e-18, -3.31913772e-02],
       [ 4.08572328e-01,  2.79321984e-01,  4.07349866e-03,
        -1.25749625e-17, -5.56408959e-06, -2.04326902e-01,
        -1.39688842e-01, -3.52247602e-02, -4.93473690e-18,
         2.26893293e-02, -2.04245426e-01, -1.39633141e-01,
        -3.52299850e-02, -2.73714419e-18, -2.26914079e-02],
       [ 5.95842407e-03,  4.07349866e-03,  5.94059627e-05,
        -1.83387259e-19, -8.11440302e-08, -2.97980613e-03,
        -2.03715549e-03, -5.13701111e-04, -7.19658506e-20,
         3.30890363e-04, -2.97861793e-03, -2.03634317e-03,
        -5.13777307e-04, -3.99172061e-20, -3.30920675e-04],
       [-1.83937606e-17, -1.25749625e-17, -1.83387259e-19,
         5.66119714e-34,  2.50493058e-22,  9.19871429

In [14]:
f_b = h_test.id.displayFockMatrix("beta")
f_a = h_test.id.displayFockMatrix("alpha")

In [15]:
f_cs = (f_a + f_b)/2

In [16]:
a = h_test.id.getDensityMatrix("alpha")
b = h_test.id.getDensityMatrix("beta")
p = (a + b)/2

In [17]:
c1, c = eigh(f_a, h_test.id.overlap)
c_inv = np.linalg.inv(c)
c_inv

array([[-7.58405463e-01, -7.94683425e-01, -3.06750457e-01,
        -1.27168923e-17, -3.08560854e-06, -7.54336437e-01,
        -7.94999942e-01,  1.47070071e-01, -1.90738239e-17,
         2.83932557e-01, -7.54337590e-01, -7.94999972e-01,
         1.47065917e-01, -1.78931461e-18, -2.83931532e-01],
       [ 5.59310915e-01,  3.22414727e-01, -3.62159382e-01,
        -3.00090498e-17,  4.19792534e-05, -2.78792245e-01,
        -1.67906146e-01, -4.18501181e-01, -3.04469997e-17,
         2.80649131e-02, -2.78885707e-01, -1.67960790e-01,
        -4.18495325e-01, -2.88952079e-17, -2.79911033e-02],
       [ 4.46536786e-05,  2.96757119e-05, -2.99489374e-05,
        -1.76528670e-16, -3.34963986e-01, -3.95587210e-01,
        -3.08330103e-01, -1.77905968e-03, -3.84554902e-16,
        -3.37655562e-01,  3.95544688e-01,  3.08286297e-01,
         1.71174132e-03, -3.63012183e-17, -3.37650518e-01],
       [-7.36245457e-02,  4.75716841e-01, -3.17088979e-01,
         6.41773236e-16, -5.40939784e-06, -1.17897895

In [18]:
d = c_inv.dot(p).dot(c_inv.T)
o1, o = eigh(d)
o = o[:, ::-1]
j = o.T.dot(d).dot(o)
j[abs(j) < 1e-10] = 0
j

array([[1. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0.5, 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
        0. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. , 0. ,
       

In [25]:
f_trans = c.T.dot(f_a).dot(c)
f_trans

array([[-6.53034051e-01,  1.50053581e-16, -8.20741045e-17,
         3.38271078e-17,  6.59194921e-17,  1.38777878e-16,
        -1.22601193e-17,  2.77555756e-17, -4.33680869e-17,
         1.38777878e-16, -4.13172693e-19, -2.45189161e-18,
         5.55111512e-17,  3.33066907e-16,  0.00000000e+00],
       [ 1.45662562e-16, -1.48958244e-01,  1.86204947e-16,
         1.51110678e-16,  6.57297567e-17, -2.53920149e-16,
        -1.43542394e-17,  1.45933612e-16, -1.52303300e-16,
         3.59521440e-16, -8.41363482e-19, -1.45351247e-17,
        -1.05384451e-16,  2.58473798e-16, -3.26128013e-16],
       [-2.88397778e-17,  2.50342282e-16,  1.55568322e-01,
         1.69569220e-16, -2.91541964e-16,  8.32667268e-16,
        -1.45017327e-16,  1.97758476e-16,  2.44379170e-16,
        -1.76941795e-16,  1.11781261e-16, -3.62596499e-17,
         6.17561557e-16, -1.38777878e-16, -8.32667268e-17],
       [ 5.55111512e-17,  2.82326246e-16,  3.45752073e-16,
         4.68059215e-01, -3.79253920e-16,  1.56125113

In [139]:
a_trans = c_inv.dot(a).dot(c_inv.T)
a_no = o.T.dot(a_trans).dot(o)
a_no[abs(a_no) < 1e-10] = 0
a_no


array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0.

In [312]:
c_inv = np.array([[1,2],[3,4]])
d_inv = np.array([[5,6],[7,8]])
total_inverse = np.einsum("pq, qr-> pr", d_inv, c_inv, optimize=True)
total_inverse

array([[23, 34],
       [31, 46]])

In [313]:
c_inv.dot(d_inv)

array([[19, 22],
       [43, 50]])

In [65]:
y = eigh(p, h_test.id.overlap)

In [99]:
n = np.array([[1,2], [2,3]])
v = eigh(n)
v

(array([-0.23606798,  4.23606798]),
 array([[-0.85065081,  0.52573111],
        [ 0.52573111,  0.85065081]]))

In [101]:
v[1].dot(n).dot(v[1].T)

array([[-2.36067977e-01, -3.46944695e-16],
       [-2.22044605e-16,  4.23606798e+00]])

In [28]:
o.T.dot(o)

array([[ 1.00000000e+00, -3.27022480e-17,  2.31002825e-17,
        -1.62630326e-18,  6.94567017e-19, -1.57209315e-18,
        -1.08420217e-18, -4.33334237e-33, -7.01038500e-32,
        -4.04445288e-33, -1.41962722e-18, -9.21571847e-19,
        -1.01643954e-19,  3.90312782e-18, -3.25260652e-19],
       [-3.27022480e-17,  1.00000000e+00, -6.81963166e-17,
        -7.63278329e-17,  1.29020059e-17, -1.08420217e-17,
        -1.38777878e-17,  3.54371110e-32,  3.69778549e-31,
         5.23852945e-32,  1.25767452e-17,  3.46944695e-18,
         9.75781955e-18, -5.46437895e-17, -8.41340886e-17],
       [ 2.31002825e-17, -6.81963166e-17,  1.00000000e+00,
         4.29344060e-17, -3.37457926e-17, -9.97465999e-18,
         1.38777878e-17,  3.15852511e-32, -3.51289622e-31,
        -1.04770589e-31,  1.89735380e-19,  1.73472348e-18,
         6.50521303e-18,  5.33427469e-17,  2.25514052e-17],
       [-1.62630326e-18, -7.63278329e-17,  4.29344060e-17,
         1.00000000e+00, -3.33934269e-17,  8.32667268